# AST Testing Techniques (Part 6 of 6)

**Prerequisites:** This notebook assumes you've completed parts 1-5 of the AST guide series:
- Part 1: AST Core Concepts
- Part 2: Essential Node Types
- Part 3: Visitor Patterns
- Part 4: Scope Tracking
- Part 5: Type Annotations

This final part covers advanced testing techniques for AST code, including programmatic AST creation, building test cases, and debugging strategies.

## Testing Your AST Code

### 24. Create Test ASTs Programmatically

Sometimes it's easier to build AST nodes directly rather than parsing code strings. This is especially useful for testing edge cases or when you want to test your visitor with specific node structures. The key is remembering to call `ast.fix_missing_locations()` to add required line number information.

In [1]:
import ast

# Building AST nodes programmatically for testing


def create_function_node(name, params, return_type=None):
    """Create a FunctionDef node programmatically."""
    # Create parameter list
    args_list = []
    for param_name, param_type in params:
        arg = ast.arg(arg=param_name, annotation=None)
        if param_type:
            arg.annotation = ast.Name(id=param_type, ctx=ast.Load())
        args_list.append(arg)

    # Create arguments object
    arguments = ast.arguments(
        posonlyargs=[], args=args_list, vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]
    )

    # Create function body (just pass for now)
    body = [ast.Pass()]

    # Create return annotation if provided
    returns = ast.Name(id=return_type, ctx=ast.Load()) if return_type else None

    # Create the function node
    func_node = ast.FunctionDef(
        name=name, args=arguments, body=body, decorator_list=[], returns=returns, type_comment=None
    )

    return func_node


def create_class_with_method():
    """Create a complete class with a method."""
    # Create method
    method = create_function_node("add", [("self", None), ("a", "int"), ("b", "int")], "int")

    # Create class
    class_node = ast.ClassDef(name="Calculator", bases=[], keywords=[], body=[method], decorator_list=[])

    return class_node


# Test creating a class
class_node = create_class_with_method()
print("Created class node:")
print(ast.dump(class_node, indent=2))

Created class node:
ClassDef(
  name='Calculator',
  body=[
    FunctionDef(
      name='add',
      args=arguments(
        args=[
          arg(arg='self'),
          arg(
            arg='a',
            annotation=Name(id='int', ctx=Load())),
          arg(
            arg='b',
            annotation=Name(id='int', ctx=Load()))]),
      body=[
        Pass()],
      returns=Name(id='int', ctx=Load()))])


In [2]:
def create_test_module():
    """Create a complete module for testing."""
    # Create an assignment: x = 5
    assign = ast.Assign(
        targets=[ast.Name(id="x", ctx=ast.Store())], value=ast.Constant(value=5), type_comment=None
    )

    # Create a function call: print(x)
    call = ast.Expr(
        value=ast.Call(
            func=ast.Name(id="print", ctx=ast.Load()), args=[ast.Name(id="x", ctx=ast.Load())], keywords=[]
        )
    )

    # Create the module
    module = ast.Module(body=[assign, call], type_ignores=[])

    # CRITICAL: Fix missing locations
    ast.fix_missing_locations(module)

    return module


# Test the programmatically created AST
module = create_test_module()

# You can compile and execute it!
code = compile(module, "[test]", "exec")
print("Executing programmatically created AST:")
exec(code)  # Will print: 5

Executing programmatically created AST:
5


In [3]:
# Or analyze it with your visitor
class TestVisitor(ast.NodeVisitor):
    def visit_Assign(self, node):
        print(f"Found assignment at line {getattr(node, 'lineno', 'unknown')}")
        self.generic_visit(node)

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            print(f"Found call to {node.func.id} at line {getattr(node, 'lineno', 'unknown')}")
        self.generic_visit(node)


visitor = TestVisitor()
visitor.visit(module)

Found assignment at line 1
Found call to print at line 1


In [4]:
# Create more complex test cases
def test_edge_cases():
    """Create AST nodes for edge cases."""
    # Empty function
    empty_func = ast.FunctionDef(
        name="empty",
        args=ast.arguments(
            posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]
        ),
        body=[ast.Pass()],
        decorator_list=[],
        returns=None,
    )

    # Function with only *args and **kwargs
    variadic_func = ast.FunctionDef(
        name="variadic",
        args=ast.arguments(
            posonlyargs=[],
            args=[],
            vararg=ast.arg(arg="args", annotation=None),
            kwonlyargs=[],
            kw_defaults=[],
            kwarg=ast.arg(arg="kwargs", annotation=None),
            defaults=[],
        ),
        body=[ast.Pass()],
        decorator_list=[],
        returns=None,
    )

    # Test module with edge cases
    module = ast.Module(body=[empty_func, variadic_func], type_ignores=[])
    ast.fix_missing_locations(module)

    return module


edge_cases = test_edge_cases()
print("Edge case AST:")
print(ast.dump(edge_cases, indent=2))

Edge case AST:
Module(
  body=[
    FunctionDef(
      name='empty',
      args=arguments(),
      body=[
        Pass()]),
    FunctionDef(
      name='variadic',
      args=arguments(
        vararg=arg(arg='args'),
        kwarg=arg(arg='kwargs')),
      body=[
        Pass()])])


In [5]:
# Verify the created AST matches parsed code
original_code = "x = 5\nprint(x)"
parsed_tree = ast.parse(original_code)
created_tree = create_test_module()

print("\nParsed AST:")
print(ast.dump(parsed_tree, indent=2))

print("\nCreated AST:")
print(ast.dump(created_tree, indent=2))
# They should be structurally identical!


Parsed AST:
Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=5)),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='x', ctx=Load())]))])

Created AST:
Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=5)),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='x', ctx=Load())]))])


### 25. Use Small Examples

When debugging AST issues, always start with the smallest possible example that reproduces your problem. This makes it much easier to understand what's happening and to test your fixes. Build up complexity gradually once the simple case works.

First, let's define a helper function for debugging AST patterns:

In [6]:
import ast


def debug_pattern(pattern_name, code_snippet):
    """Helper for debugging specific AST patterns."""
    print(f"\n=== Debugging: {pattern_name} ===")
    print(f"Code: {code_snippet}")

    try:
        tree = ast.parse(code_snippet)
        print("AST Structure:")
        print(ast.dump(tree, indent=2))

        # Walk through all nodes and show their types
        print("\nNode types present:")
        node_types = set()
        for node in ast.walk(tree):
            node_types.add(type(node).__name__)
        for node_type in sorted(node_types):
            print(f"  - {node_type}")

        return tree
    except SyntaxError as e:
        print(f"Syntax Error: {e}")
        return None

In [7]:
# Debug the specific pattern we've been working with
debug_pattern(
    "Instance method call pattern",
    """calc = Calculator()
calc.add(1, 2)""",
)


=== Debugging: Instance method call pattern ===
Code: calc = Calculator()
calc.add(1, 2)
AST Structure:
Module(
  body=[
    Assign(
      targets=[
        Name(id='calc', ctx=Store())],
      value=Call(
        func=Name(id='Calculator', ctx=Load()))),
    Expr(
      value=Call(
        func=Attribute(
          value=Name(id='calc', ctx=Load()),
          attr='add',
          ctx=Load()),
        args=[
          Constant(value=1),
          Constant(value=2)]))])

Node types present:
  - Assign
  - Attribute
  - Call
  - Constant
  - Expr
  - Load
  - Module
  - Name
  - Store


In [8]:
# Start simple, then add complexity
debugging_sequence = [
    # Level 1: Simplest possible case
    ("Simple assignment", "x = 5"),
    ("Simple call", "foo()"),
    # Level 2: One step more complex
    ("Constructor call", "calc = Calculator()"),
    ("Method call", "calc.add()"),
    # Level 3: The actual pattern
    (
        "Full pattern",
        """calc = Calculator()
calc.add(1, 2)""",
    ),
    # Level 4: Edge cases
    (
        "Reassignment",
        """calc = Calculator()
calc = Processor()
calc.process()""",
    ),
    ("Chained calls", "get_calc().add(1, 2)"),
    (
        "Nested scopes",
        """def outer():
    calc = Calculator()
    def inner():
        calc.add(1, 2)""",
    ),
]

for name, code in debugging_sequence:
    debug_pattern(name, code)


=== Debugging: Simple assignment ===
Code: x = 5
AST Structure:
Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=5))])

Node types present:
  - Assign
  - Constant
  - Module
  - Name
  - Store

=== Debugging: Simple call ===
Code: foo()
AST Structure:
Module(
  body=[
    Expr(
      value=Call(
        func=Name(id='foo', ctx=Load())))])

Node types present:
  - Call
  - Expr
  - Load
  - Module
  - Name

=== Debugging: Constructor call ===
Code: calc = Calculator()
AST Structure:
Module(
  body=[
    Assign(
      targets=[
        Name(id='calc', ctx=Store())],
      value=Call(
        func=Name(id='Calculator', ctx=Load())))])

Node types present:
  - Assign
  - Call
  - Load
  - Module
  - Name
  - Store

=== Debugging: Method call ===
Code: calc.add()
AST Structure:
Module(
  body=[
    Expr(
      value=Call(
        func=Attribute(
          value=Name(id='calc', ctx=Load()),
          attr='add',
          ctx=Load(

In [9]:
# Create minimal test visitor for specific pattern
class MinimalPatternTracker(ast.NodeVisitor):
    """Minimal visitor to demonstrate variable type tracking."""

    def __init__(self):
        self.assignments = {}
        self.calls = []

    def visit_Assign(self, node):
        # Track: var = ClassName()
        if (
            len(node.targets) == 1
            and isinstance(node.targets[0], ast.Name)
            and isinstance(node.value, ast.Call)
            and isinstance(node.value.func, ast.Name)
        ):
            var_name = node.targets[0].id
            class_name = node.value.func.id
            self.assignments[var_name] = class_name
            print(f"Tracked: {var_name} = {class_name}()")

        self.generic_visit(node)

    def visit_Call(self, node):
        # Track: var.method()
        if isinstance(node.func, ast.Attribute) and isinstance(node.func.value, ast.Name):
            var_name = node.func.value.id
            method_name = node.func.attr

            # Try to resolve
            if var_name in self.assignments:
                class_name = self.assignments[var_name]
                resolved = f"{class_name}.{method_name}"
                print(f"Resolved: {var_name}.{method_name} -> {resolved}")
                self.calls.append(resolved)
            else:
                print(f"Unresolved: {var_name}.{method_name}")

        self.generic_visit(node)

In [10]:
# Test the minimal pattern tracker
test_code = """
calc = Calculator()
calc.add(1, 2)
proc = Processor()
proc.run()
unknown.method()
"""

tree = ast.parse(test_code)
tracker = MinimalPatternTracker()
tracker.visit(tree)
print(f"\nFinal calls: {tracker.calls}")

Tracked: calc = Calculator()
Resolved: calc.add -> Calculator.add
Tracked: proc = Processor()
Resolved: proc.run -> Processor.run
Unresolved: unknown.method

Final calls: ['Calculator.add', 'Processor.run']


In [11]:
# Pro tip: Use assertions for test cases
def test_ast_pattern(code, expected_call):
    """Test helper with assertions."""
    tree = ast.parse(code)
    tracker = MinimalPatternTracker()
    tracker.visit(tree)

    assert expected_call in tracker.calls, f"Expected {expected_call}, got {tracker.calls}"
    print(f"✓ Test passed: {expected_call}")


# Run tests
test_ast_pattern("calc = Calculator()\ncalc.add()", "Calculator.add")
test_ast_pattern("p = Processor()\np.run()", "Processor.run")

Tracked: calc = Calculator()
Resolved: calc.add -> Calculator.add
✓ Test passed: Calculator.add
Tracked: p = Processor()
Resolved: p.run -> Processor.run
✓ Test passed: Processor.run


## Summary: What You Need to Master

For implementing scope-aware variable tracking in static analysis tools:
1. **Assignment detection** (ast.Assign, ast.AnnAssign)
2. **Scope tracking** (function_stack management)
3. **Variable type storage** (scoped dictionary: "scope.var" → "Type")
4. **Call resolution** (looking up variable types during ast.Call processing)

For building more advanced static analysis capabilities:
1. **Import tracking** (ast.Import, ast.ImportFrom)
2. **Class attributes** (assignments within ClassDef body)
3. **Type narrowing** (isinstance checks in if statements)
4. **Complex annotations** (Optional, Union, generics)

This comprehensive guide covers the essential AST concepts needed for building sophisticated code analysis tools. By understanding these patterns and testing techniques, you'll be well-equipped to implement complex static analysis features with confidence.

## Conclusion

This completes the 6-part AST guide series. You now have a comprehensive understanding of:

1. **Core AST concepts** and how Python's abstract syntax trees work
2. **Essential node types** for analyzing Python code structure  
3. **Visitor patterns** for traversing and analyzing AST nodes
4. **Scope tracking** techniques for managing variable visibility
5. **Type annotation** analysis for understanding code contracts
6. **Testing techniques** for debugging and validating AST code

With these skills, you're ready to tackle complex code analysis tasks, build sophisticated static analysis tools, and contribute effectively to projects that work with Python's AST.